In [1]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score, roc_auc_score
import json

In [2]:
#X = pd.read_csv('./IU_Xray/clean_data.csv')

#Y = pd.read_csv('./IU_Xray/labeled_reports.csv')

#Y['report_id'] = X['uid'].apply(lambda x: 'CXR'+str(x))

#Y = Y[['report_id','Enlarged Cardiomediastinum', 'Cardiomegaly',
#       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
#       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
#       'Fracture', 'Support Devices', 'No Finding']]#.to_csv('chexbert_labels.csv')

#Y = Y.fillna(0)
#Y.replace(-1,1)
#Y.to_csv('./IU_Xray/chexbert_labels.csv')

In [3]:
labels_path = '../../IU_data/chexbert_labeling/'

In [4]:
X = pd.read_csv(labels_path+'report_impressions.csv')

Y = pd.read_csv(labels_path+'labeled_reports.csv')

Y['report_id'] = X['Id']

Y = Y[['report_id','Enlarged Cardiomediastinum', 'Cardiomegaly',
      'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
      'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
      'Fracture', 'Support Devices', 'No Finding']]#.to_csv('chexbert_labels.csv')

Y = Y.fillna(0)
Y = Y.replace(-1,1)
Y.to_csv(labels_path+'chexbert_labels.csv')

In [5]:
path='/media/zaheer/Data/Image_Text_Datasets/IU_Xray/latest/One_Image_norm_abnorm_split/r2gen_annotations/'

In [6]:
with open(path+'images_features_2d_r2gen.pickle','rb') as features:
#with open(path+'densenet_report_features.pkl','rb') as features:
    data=pickle.load(features)

In [7]:
data['CXR1'][0].shape

(1, 7, 7, 1024)

In [8]:
feature_shape = 1024

In [9]:
def clean_r2gen_annotations():
    
    tags = pd.read_csv('./IU_Xray/chexbert_labels.csv')
    
    reports = tags['report_id'].tolist()
    
    #print(reports)
    
    with open(path+'annotation.json', 'rb') as f:
        full_records = json.load(f)

    splits=['train','val','test']
    
    new_images_records={}
    
    for s in splits:
        new_records=[]
        records=full_records[s]
        for r in records:
            for image in r['image_path']:
                if image.split('_')[0] in reports: 
                    new_records.append(r)
                    break
                
        new_images_records[s]=new_records
        
    with open(path+'new_annotation.json', 'w') as f:
        json.dump(new_images_records , f, ensure_ascii = False )
        
    return new_images_records

#annot = clean_r2gen_annotations()

def fetch_tags(image_id, tags):

    new_tags=[]
    image_ids =[]

    for image in image_id:
        
        
        tag=(tags[tags.report_id == image.split('_')[0]][['Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
      'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
      'Fracture', 'Support Devices', 'No Finding']])
        if tag.shape[0] != 0:
            tag = tag.values.tolist()[0]
            image_ids.append(image)
            new_tags.append(tag)

    return image_ids, new_tags


def convert_r2gen_annotations():
    with open(path+'new_annotation.json', 'rb') as f:
        full_records = json.load(f)

    splits=['train','val','test']
    
    new_images_records={}
    
    for s in splits:
        images=[]
        records=full_records[s]
        for r in records:
            for image in r['image_path']:
                images.append(image)
                
        new_images_records[s]=images
    return new_images_records

convert_r2gen_annotations()

def load_preprocess_data():
    tags = pd.read_csv(labels_path+'chexbert_labels.csv')
    converted_records = convert_r2gen_annotations()
    x_train = converted_records['train']
    x_test = converted_records['test']
    x_val = converted_records['val']
    #print(train.shape, train.columns)

    x_train, train_tags=fetch_tags(x_train,tags)
    #print(train_tags)
    #y_train = mlb.fit_transform([train_tags])
    y_train=np.array(train_tags)
    #print(y_train)

    #print(y_train.head())
    
    
    x_test, test_tags=fetch_tags(x_test,tags)
    y_test = np.array(test_tags)
    
    
    x_val, val_tags=fetch_tags(x_val,tags)
    y_val = np.array(val_tags)
    return x_train,y_train,x_test,y_test, x_val,y_val



In [10]:
def load_features(split):
    features = []#np.empty((0,7,7,1024))
    
    for image in split:
        #print(image)
        report_id = image.split('_')[0]
        image_id = image.split('.')[0].split('/')[1]
        features.append(data[report_id][int(image_id)])
    features=np.array(features)
    features=np.reshape(features,(-1,49,feature_shape))
    
    return features

In [11]:
def class_model(n):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(49,feature_shape)))
    
    model.add(layers.GlobalAveragePooling1D())
    #model.add(layers.Dense(128,activation='relu'))
    #model.add(layers.LeakyReLU(alpha=0.05))
    #model.add(layers.Dense(64,activation='relu'))
    #model.add(layers.Dropout(0.4))
    model.add(layers.Dense(n, activation="sigmoid"))

    model.summary()
    
    return model



In [12]:
images_train,y_train,images_test,y_test, images_val,y_val=load_preprocess_data()

In [13]:

train_features=load_features(images_train)
test_features=load_features(images_test)
val_features=load_features(images_val)

In [14]:
y_train.shape

(4126, 14)

In [15]:
model=class_model(14)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling1d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                14350     
Total params: 14,350
Trainable params: 14,350
Non-trainable params: 0
_________________________________________________________________


In [16]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()
])

model.fit(train_features, y_train, epochs=100, callbacks=[earlyStopping],validation_data=(val_features, y_val), batch_size=16)

test_tags=model.predict(test_features)
val_tags=model.predict(val_features)
train_tag = model.predict(train_features)

Train on 4126 samples, validate on 592 samples
Epoch 1/100
4126/4126 [==============================] - 2s 412us/sample - loss: 0.1795 - precision: 0.6955 - recall: 0.6383 - auc: 0.8946 - val_loss: 0.1374 - val_precision: 0.7270 - val_recall: 0.6511 - val_auc: 0.9335
Epoch 2/100
4126/4126 [==============================] - 1s 188us/sample - loss: 0.1415 - precision: 0.7524 - recall: 0.6294 - auc: 0.9258 - val_loss: 0.1319 - val_precision: 0.7443 - val_recall: 0.6573 - val_auc: 0.9389
Epoch 3/100
4126/4126 [==============================] - 1s 189us/sample - loss: 0.1371 - precision: 0.7641 - recall: 0.6320 - auc: 0.9309 - val_loss: 0.1291 - val_precision: 0.7559 - val_recall: 0.6511 - val_auc: 0.9409
Epoch 4/100
4126/4126 [==============================] - 1s 189us/sample - loss: 0.1344 - precision: 0.7676 - recall: 0.6287 - auc: 0.9342 - val_loss: 0.1269 - val_precision: 0.7734 - val_recall: 0.6433 - val_auc: 0.9431
Epoch 5/100
4126/4126 [==============================] - 1s 192us/sam

In [17]:
#test_tags=np.where(test_tags<0.50,0,1)
test_tags

array([[3.3833385e-03, 1.8214971e-02, 4.4425726e-02, ..., 4.6948195e-03,
        2.1142751e-02, 8.2641602e-01],
       [1.0030299e-02, 7.2295278e-02, 7.8657329e-02, ..., 5.9010386e-03,
        7.9286695e-03, 6.6443324e-01],
       [2.6425719e-04, 8.4444880e-04, 9.5446408e-03, ..., 1.2635887e-03,
        3.1249523e-03, 9.8386562e-01],
       ...,
       [5.0853491e-03, 2.1627516e-02, 1.0323793e-02, ..., 2.3558140e-03,
        9.1944337e-03, 9.4369447e-01],
       [9.6302629e-03, 1.5771538e-02, 4.3721288e-02, ..., 5.1297247e-03,
        1.8338263e-03, 7.4064279e-01],
       [9.5665753e-03, 5.6407660e-02, 5.9154421e-02, ..., 3.8816333e-03,
        1.0374546e-02, 7.4696958e-01]], dtype=float32)

In [18]:
#val_tags=np.where(val_tags<0.50,0,1)
val_tags

array([[0.0126442 , 0.14028746, 0.1144709 , ..., 0.03314736, 0.01262876,
        0.6204095 ],
       [0.00951898, 0.08002225, 0.16328329, ..., 0.00737441, 0.00247383,
        0.57513255],
       [0.00702068, 0.09686702, 0.13626158, ..., 0.0145683 , 0.00466797,
        0.5832921 ],
       ...,
       [0.02296054, 0.2169556 , 0.17006353, ..., 0.01215476, 0.00761154,
        0.40859276],
       [0.02245969, 0.94433063, 0.19485441, ..., 0.01793998, 0.09112912,
        0.07793838],
       [0.00580779, 0.03402773, 0.07120609, ..., 0.00333947, 0.01448455,
        0.7839531 ]], dtype=float32)

In [19]:
y_test_0=np.where(y_test==-1.0,0,y_test)

In [20]:
y_test_1=np.where(y_test==-1.0,1,y_test)

In [21]:
#val_tags=[i[0] for i in val_tags.tolist()]
#val_tags=pd.Series(val_tags)

In [22]:
#print(f1_score(y_test,test_tags))

In [23]:
#np.max(y_test_1[:,cidx])

In [24]:
for cidx in range(y_test.shape[1]):
    print(roc_auc_score(pd.Series(y_test_1[:,cidx]), pd.Series(test_tags[:,cidx])))

0.6030769230769231
0.8571297114252062
0.7995568181818181
0.6052479815455594
0.9125843931543413
0.6752346416382252
0.813738344988345
0.826288302844942
0.6849801249290176
0.950247668997669
0.7791027825099376
0.41198182850653037
0.7321122273952463
0.8148233074899627


In [25]:
# for cidx in range(y_test.shape[1]):
#     print(f1_score(pd.Series(y_test_0[:,cidx]), pd.Series(test_tags[:,cidx])))

In [26]:
m=tf.keras.metrics.Recall()
m.update_state(y_test,test_tags)
m.result().numpy()

0.7271318

In [27]:
import pickle

all_tags={}
all_reports = []
test_true = np.array(y_test)
for idx,image in enumerate(images_test):
    all_tags[image.split('_')[0]] = test_tags[idx,:]
    #all_tags[image] = test_tags[idx,:]
    #all_tags[image.split('_')[0]]=test_true[idx,:]
    all_reports.append(image.split('_')[0])

val_true = np.array(y_val)
for idx,image in enumerate(images_val):
    all_tags[image.split('_')[0]] = val_tags[idx,:]
    #all_tags[image] = val_tags[idx,:]
    #all_tags[image.split('_')[0]]=val_true[idx,:]
    all_reports.append(image.split('_')[0])
    
train_true=np.array(y_train)
for idx,image in enumerate(images_train):
    # all_tags[image.split('_')[0]]=train_true[idx,:]
    all_tags[image.split('_')[0]] = train_tag[idx,:]
    #all_tags[image] = train_tag[idx,:]
    all_reports.append(image.split('_')[0])


all_reports = list(set(all_reports))

#print(all_tags['CXR3614'])
with open(path+'chexbert_chex_r2gen.pkl','wb') as file:
#with open(path+'chexbert_dense_r2gen.pkl','wb') as file:
    pickle.dump(all_tags, file, protocol=2)
    

In [28]:
import numpy as np    
arr = np.empty((0,3), int)
print("Empty array:")
print(arr)
arr = np.append(arr, np.array([[10,20,30]]), axis=0)
arr = np.append(arr, np.array([[40,50,60]]), axis=0)
print("After adding two new arrays:")
print(arr)

Empty array:
[]
After adding two new arrays:
[[10 20 30]
 [40 50 60]]


In [29]:
B = 2*[[3,4]]
A = arr
print(B)
np.insert(A,A.shape[1],B,axis=1)

[[3, 4], [3, 4]]


array([[10, 20, 30,  3,  3],
       [40, 50, 60,  4,  4]])